In [1]:
import os

In [2]:
%pwd

'c:\\Data\\projects\\Machine-Learning-Project_with_MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Data\\projects\\Machine-Learning-Project_with_MLFlow'

In [6]:
%pip install dagshub

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   -------------- ------------------------- 4.5/12.5 MB 20.7 MB/s eta 0:00:01
   ---------------------------------------  12.3/12.5 MB 27.6 MB/s eta 0:00:01
   ---------------------------------------- 12.5/12.5 MB 23.7 MB/s eta 0:00:00
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10501 sha256=610bb9a18191d34edd92c37d9517a51786f42599663d1b22b6b900d29e9c6375
  Stored in directory: c:\users\abhana\appdata\local\pip\cache\wheels\db\4a\86\fdda91f8b8ebb0a70e4181dc2423b1f70c3c2d3bd1158685b5
Successfully built fusepy
  Attempting uninstall: httpcore
    Found existing instal

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.4 requires httpx>=0.25.0, but you have httpx 0.23.3 which is incompatible.


In [9]:
import os
import dagshub

dagshub.init(repo_owner='AbhilashAnand06', repo_name='Machine-Learning-Project_with_MLFlow', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Data\projects\Machine-Learning-Project_with_MLFlow\mlproj\Lib\site-packages\rich\live.py:229: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=c90844b0-0992-4946-bd0a-64e9c68aa3e8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a4136c127693c250d50920c20f5e82b4d6706bbde0c69459be496cc315eddae7




ConnectError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)

In [18]:


os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/AbhilashAnand06/Machine-Learning-Project_with_MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="AbhilashAnand06"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6d8a462f0d07d33de35fb785b2f347e620b29403"

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [20]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/AbhilashAnand06/Machine-Learning-Project_with_MLFlow.mlflow",
           
        )

        return model_evaluation_config


In [22]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [23]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [24]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-08-13 03:11:56,100: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-13 03:11:56,108: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-13 03:11:56,120: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-13 03:11:56,122: INFO: common: created directory at: artifacts]
[2024-08-13 03:11:56,123: INFO: common: created directory at: artifacts/model_evaluation]


[2024-08-13 03:11:56,718: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)'))': /AbhilashAnand06/Machine-Learning-Project_with_MLFlow.mlflow/api/2.0/mlflow/runs/create]
[2024-08-13 03:12:00,803: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)'))': /AbhilashAnand06/Machine-Learning-Project_with_MLFlow.mlflow/api/2.0/mlflow/runs/create]
[2024-08-13 03:12:08,845: WARNING: connectionpool: Retrying (Retry(total=2, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] c